In [3]:
%load_ext autoreload
%autoreload 2
from src.config import RAW_DATA_PATH
from src.DataProcessor import create_order_items
from src.IOController import create_pyspark_session

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
spark = create_pyspark_session()

Starting PySpark session. Check your terminal for detailed logging...
PySpark session sucessfully created.


In [5]:
df = spark.read.parquet(str(RAW_DATA_PATH / 'order'))
print('(#rows, #columns) =', (df.count(), len(df.columns)))
df.printSchema()

(#rows, #columns) = (3683040, 22)
root
 |-- cpf: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: string (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: string (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: string (nullable = true)
 |-- merchant_longitude: string (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: timestamp (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- order_schedule

In [7]:
tmp = create_order_items(df)
tmp.limit(5).toPandas().T

,0,1,2,3,4
order_id,dd4f8f0a-c2cb-45c6-a002-c3be6b305e5f,dd4f8f0a-c2cb-45c6-a002-c3be6b305e5f,8dd80f0b-db00-4b88-b7e2-02ca706fc5a5,8dd80f0b-db00-4b88-b7e2-02ca706fc5a5,8dd80f0b-db00-4b88-b7e2-02ca706fc5a5
name,Parmegiana de Filé de Frango (2 pessoas),Lasanha Frango (2 pessoas),Filé Mignon à Cubana,603- Pudim de leite,601-Torta Holandesa
addition,"{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}"
discount,"{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}"
quantity,1,1,1,1,1
sequence,1,3,1,4,5
unitPrice,"{""value"":""2800"",""currency"":""BRL""}","{""value"":""1800"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}"
externalId,0bcd6764fd5e466d9c04b18ac0eb69e6,a361f5eec6a44ac0817892e81bf22e80,e0e81b2027c241cca5bf60a4768800ec,4436f76edc164052aaf3c21a6567b570,c0b788e876c54f198307dc142fed7aff
totalValue,"{""value"":""2800"",""currency"":""BRL""}","{""value"":""1800"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}"
customerNote,None,None,None,None,None


In [14]:
tmp.dropna()

DataFrame[order_id: string, name: string, addition: string, discount: string, quantity: float, sequence: float, unitPrice: string, externalId: string, totalValue: string, customerNote: string, garnishItems: string, integrationId: string, totalAddition: string, totalDiscount: string]

In [44]:
x = tmp.repartition(1)#.toPandas()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o239.repartition

In [5]:
sensitive_data_columns = [
    'cpf',
    'customer_name',
]

In [6]:
for dtype, cols in dtypes.items():
    for col in cols:
        df = df.withColumn(col, df[col].cast(dtype))

In [7]:
df.printSchema()

root
 |-- cpf: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: float (nullable = true)
 |-- delivery_address_longitude: float (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: long (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: float (nullable = true)
 |-- merchant_longitude: float (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: timestamp (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- order_scheduled_date: timestamp (nullable = true)
 |-- o

In [8]:
df.limit(5).toPandas().T

,0,1,2,3,4
cpf,80532101763,43352103961,38650991217,63579726866,90617788806
customer_id,977b9a89-825f-464b-8ef6-0f453d7334c1,e969cc0d-388b-4025-9351-0db0f718d81c,e08dcc8b-f998-405e-b3f2-7107ea8958cf,6b40a7cc-de6a-4456-ada3-66d67453b88e,647469d8-e0f8-4fce-8fa9-eafd1802ccf1
customer_name,GUSTAVO,MICHELLE,VICTOR,ANNIE,DANIEL
delivery_address_city,FRANCA,SANTOS,GUARULHOS,SAO PAULO,VITORIA
delivery_address_country,BR,BR,BR,BR,BR
delivery_address_district,JARDIM ESPRAIADO,CAMPO GRANDE,JARDIM ROSSI,PARQUE SAO JORGE,JARDIM CAMBURI
delivery_address_external_id,6736655,8759216,8765930,7834087,7211683
delivery_address_latitude,-47.39,-46.34,-46.53,-46.57,-40.27
delivery_address_longitude,-20.55,-23.96,-23.44,-23.53,-20.25
delivery_address_state,SP,SP,SP,SP,ES


In [12]:
df.printSchema()

root
 |-- cpf: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: float (nullable = true)
 |-- delivery_address_longitude: float (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: long (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- addition: string (nullable = true)
 |    |    |-- discount: string (nullable = true)
 |    |    |-- quantity: float (nullable = true)
 |    |    |-- sequence: float (nullable = true)
 |    |    |-- unitPrice: string (nullable = true)
 |    |    |-- externalId: string (nullable = t

In [13]:
df.select('items').show()

+--------------------+
|               items|
+--------------------+
|[[Parmegiana de F...|
|[[Filé Mignon à C...|
|[[GRANDE 2 SABORE...|
|[[CALABRESA, {"va...|
|[[GRANDE (35CM) 8...|
|[[GRANDE 2 SABORE...|
|[[Açaí, {"value":...|
|[[ROLINHO DE CHOC...|
|[[GRANDE  2 SABOR...|
|[[Açaí, {"value":...|
|[[ASTORIAS FRIES,...|
|[[Stella Artois -...|
|[[churrasquinho m...|
|[[MAKIMONO HOT FI...|
|[[PICANHA CHEDDAR...|
|[[152 - Milk Shak...|
|[[New York Steak,...|
|[[Cerveja Heineke...|
|[[GRANDE 2 SABORE...|
|[[Executivo Grill...|
+--------------------+
only showing top 20 rows



In [14]:
df.select('order_id',explode(df['items']).alias('exploded_items')).show()

+--------------------+--------------------+
|            order_id|      exploded_items|
+--------------------+--------------------+
|dd4f8f0a-c2cb-45c...|[Parmegiana de Fi...|
|dd4f8f0a-c2cb-45c...|[Lasanha Frango (...|
|8dd80f0b-db00-4b8...|[Filé Mignon à Cu...|
|8dd80f0b-db00-4b8...|[603- Pudim de le...|
|8dd80f0b-db00-4b8...|[601-Torta Holand...|
|8dd80f0b-db00-4b8...|[513 - Guarnição ...|
|430f9887-a563-45e...|[GRANDE 2 SABORES...|
|1294917e-2247-489...|[CALABRESA, {"val...|
|1294917e-2247-489...|[CHEESE SALADA, {...|
|34755d74-781a-461...|[GRANDE (35CM) 8 ...|
|cdeeb2c8-75d7-486...|[GRANDE 2 SABORES...|
|cdeeb2c8-75d7-486...|[Guaraná Antarcti...|
|12933124-f673-4ef...|[Açaí, {"value":"...|
|3c659e61-4213-471...|[ROLINHO DE CHOCO...|
|3c659e61-4213-471...|[YAKISOBA DE VEGE...|
|fa058a2f-11a2-409...|[GRANDE  2 SABORE...|
|8bda711e-11af-460...|[Açaí, {"value":"...|
|8bda711e-11af-460...|[TRIO - Número 1,...|
|8bda711e-11af-460...|[Trio - Cheddar, ...|
|a48588a3-0f54-493...|[ASTORIAS 

In [23]:
tmp = df.select('order_id', explode(df['items']).alias('items'))
tmp.select('order_id', 'items.*').limit(5).toPandas().T

,0,1,2,3,4
order_id,dd4f8f0a-c2cb-45c6-a002-c3be6b305e5f,dd4f8f0a-c2cb-45c6-a002-c3be6b305e5f,8dd80f0b-db00-4b88-b7e2-02ca706fc5a5,8dd80f0b-db00-4b88-b7e2-02ca706fc5a5,8dd80f0b-db00-4b88-b7e2-02ca706fc5a5
name,Parmegiana de Filé de Frango (2 pessoas),Lasanha Frango (2 pessoas),Filé Mignon à Cubana,603- Pudim de leite,601-Torta Holandesa
addition,"{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}"
discount,"{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}"
quantity,1,1,1,1,1
sequence,1,3,1,4,5
unitPrice,"{""value"":""2800"",""currency"":""BRL""}","{""value"":""1800"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}"
externalId,0bcd6764fd5e466d9c04b18ac0eb69e6,a361f5eec6a44ac0817892e81bf22e80,e0e81b2027c241cca5bf60a4768800ec,4436f76edc164052aaf3c21a6567b570,c0b788e876c54f198307dc142fed7aff
totalValue,"{""value"":""2800"",""currency"":""BRL""}","{""value"":""1800"",""currency"":""BRL""}","{""value"":""0"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}","{""value"":""800"",""currency"":""BRL""}"
customerNote,None,None,None,None,None


In [24]:
tmp.show()

+--------------------+--------------------+
|            order_id|               items|
+--------------------+--------------------+
|dd4f8f0a-c2cb-45c...|[Parmegiana de Fi...|
|dd4f8f0a-c2cb-45c...|[Lasanha Frango (...|
|8dd80f0b-db00-4b8...|[Filé Mignon à Cu...|
|8dd80f0b-db00-4b8...|[603- Pudim de le...|
|8dd80f0b-db00-4b8...|[601-Torta Holand...|
|8dd80f0b-db00-4b8...|[513 - Guarnição ...|
|430f9887-a563-45e...|[GRANDE 2 SABORES...|
|1294917e-2247-489...|[CALABRESA, {"val...|
|1294917e-2247-489...|[CHEESE SALADA, {...|
|34755d74-781a-461...|[GRANDE (35CM) 8 ...|
|cdeeb2c8-75d7-486...|[GRANDE 2 SABORES...|
|cdeeb2c8-75d7-486...|[Guaraná Antarcti...|
|12933124-f673-4ef...|[Açaí, {"value":"...|
|3c659e61-4213-471...|[ROLINHO DE CHOCO...|
|3c659e61-4213-471...|[YAKISOBA DE VEGE...|
|fa058a2f-11a2-409...|[GRANDE  2 SABORE...|
|8bda711e-11af-460...|[Açaí, {"value":"...|
|8bda711e-11af-460...|[TRIO - Número 1,...|
|8bda711e-11af-460...|[Trio - Cheddar, ...|
|a48588a3-0f54-493...|[ASTORIAS 

In [161]:
df.printSchema()

root
 |-- cpf: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: float (nullable = true)
 |-- delivery_address_longitude: float (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: long (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- addition: string (nullable = true)
 |    |    |-- discount: string (nullable = true)
 |    |    |-- quantity: float (nullable = true)
 |    |    |-- sequence: float (nullable = true)
 |    |    |-- unitPrice: string (nullable = true)
 |    |    |-- externalId: string (nullable = t

In [115]:
import json
(json.loads(df[['items']].head(1)[0][0])[0])

{'name': 'Parmegiana de Filé de Frango (2 pessoas)',
 'addition': {'value': '0', 'currency': 'BRL'},
 'discount': {'value': '0', 'currency': 'BRL'},
 'quantity': 1.0,
 'sequence': 1,
 'unitPrice': {'value': '2800', 'currency': 'BRL'},
 'externalId': '0bcd6764fd5e466d9c04b18ac0eb69e6',
 'totalValue': {'value': '2800', 'currency': 'BRL'},
 'customerNote': None,
 'garnishItems': [{'name': 'COM Arroz branco',
   'addition': {'value': '0', 'currency': 'BRL'},
   'discount': {'value': '0', 'currency': 'BRL'},
   'quantity': 1.0,
   'sequence': 2,
   'unitPrice': {'value': '0', 'currency': 'BRL'},
   'categoryId': '13HDH',
   'externalId': '384bd2b4eb7d454d8e0274e7d590ab4f',
   'totalValue': {'value': '0', 'currency': 'BRL'},
   'categoryName': 'PERSONALIZAR',
   'integrationId': None}],
 'integrationId': 'PMFR',
 'totalAddition': {'value': '0', 'currency': 'BRL'},
 'totalDiscount': {'value': '0', 'currency': 'BRL'}}

In [ ]:
.

In [42]:
df['items'].cast('struct')

ParseException: 
DataType struct is not supported.(line 1, pos 0)

== SQL ==
struct
^^^


In [52]:
df.withColumn('items', explode(df['items'].cast('array<Struct>')))

ParseException: 
DataType struct is not supported.(line 1, pos 6)

== SQL ==
array<Struct>
------^^^


In [102]:
from_json()

TypeError: from_json() missing 2 required positional arguments: 'col' and 'schema'